# Let's get you started with asynchronous programming | Ryan Varley

**PyData Global 2024 | 3rd December 2024**

This notebook implements some of the code from the slides. Refer to README.md for instructions on setting up the repository and the test server.

The code differs slightly from what was shown during the talk, as the presentation used a real server. This code has been adapted to use a local test server, which somewhat emulates the behaviour of the real one but does not connect to a real database or similar.

For the talk itself, see [this post](https://blog.ryanvarley.com/p/pydata-global-lets-get-you-started).

Remember, we are in a notebook environment so we dont need to use `asyncio.run` here 😉

*import statements are often copied within code blocks to help you copy and paste, its not normal practice 🙂*

In [1]:
import requests
import aiohttp
import asyncio

In [2]:
API_URL = "http://0.0.0.0:8000"

In [3]:
# Get list of video ids from the server to use later
# If this fails check the server is running (see README.md)
response = requests.get(
    f"{API_URL}/videos?n=100",
)
response.raise_for_status()
valid_ids = response.json()
video_id = valid_ids[0]
video_id

'student'

## Normal (synchronous) version

Lets make a standard get request

In [4]:
import requests


def get_video(video_id):
    response = requests.get(
        f"{API_URL}/videos/{video_id}",
    )
    response.raise_for_status()
    return response.json()


get_video(video_id=video_id)

{'id': 'student',
 'title': 'Security help surface full population power show.',
 'description': 'Us executive blood wrong. Report whom green.',
 'upload_date': '2021-02-05',
 'views': 925068,
 'warnings': []}

## Asynchronous version

In [5]:
import aiohttp


async def get_video_async(video_id):
    async with aiohttp.ClientSession() as session:
        async with session.get(f"{API_URL}/videos/{video_id}") as response:
            response.raise_for_status()
            return await response.json()


video = await get_video_async(video_id=video_id)
video

{'id': 'student',
 'title': 'Security help surface full population power show.',
 'description': 'Us executive blood wrong. Report whom green.',
 'upload_date': '2021-02-05',
 'views': 925068,
 'warnings': []}

await blocks within the same coroutine. So this next function will take 4x as one as one call and behaves the same as the synchronous version running one at a time

In [6]:
async def multiple_calls():
    await get_video_async(valid_ids[0])
    await get_video_async(valid_ids[1])
    await get_video_async(valid_ids[2])
    await get_video_async(valid_ids[3])


await multiple_calls()

## Let’s make 100 calls to an endpoint (with async)

Here we introduce `asyncio.gather` that allows us to combine multiple awaitables into one awaitable where they will run concurrently.

We redefine get_video_async to accept a session.

In [7]:
import asyncio


async def get_video_async(video_id, session):
    async with session.get(f"{API_URL}/videos/{video_id}") as response:
        response.raise_for_status()
        return await response.json()


async def fetch_all_videos(video_ids, limit_per_host=10):
    connector = aiohttp.TCPConnector(limit_per_host=limit_per_host)
    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = [get_video_async(video_id, session) for video_id in video_ids]
        return await asyncio.gather(*tasks)

First no limit, lets overwhelm the server

In [8]:
videos = await fetch_all_videos(video_ids=valid_ids, limit_per_host=100)
len(videos)

100

Now lets add a limit so we arent making too many requests at the same time (remember the default is 100!)

In [9]:
videos = await fetch_all_videos(video_ids=valid_ids, limit_per_host=10)
len(videos)

100